In [ ]:
import pandas as pd  #R-like wrapper around numpy (python numerics library)
import matplotlib as mpl  #Loading the base library incase we need it
import matplotlib.pyplot as plt  #The plot functionality of matplotlib
from matplotlib.patches import Rectangle  #A patch is a type of 'Artist'; we just want the rectangle
import pdb  #Python debug
import numpy as np
%matplotlib inline

In [ ]:
def load_reaction_grid(path):
    # Read in the data file with settings applicable to Avida's output style
    # We're also going to *infer* what the data types are.  This is new in Pandas 0.21.
    # (If we don't infer the type, they aren't treated as numerics for numbers, for example.)
    data = pd.read_csv(path, 
                       comment='#', skip_blank_lines=True, 
                       delimiter=' ', header=None).infer_objects()

    nrows,ncols = data.shape  # We're interested in the number of columns for our column labels
    col_names = ['update','reaction']  # The first two columns are the update number and reaction name

    # Let's build a label for the rest of the columns
    for cell_id, col in enumerate(range(2,ncols)):
        # range(start,end) provides a list of integers from start to end-1
        # Here we're interested in column 2 to the last column
        # enumerate(list) gives us a tuple for each element: an index and the value of the element at the index
        col_names.append(f'cell_{cell_id}')  #f'cell_{cell_id}' is the same as 'cell_{x}'.format(x=cell_id)
    data.columns = col_names
    
    return data


def verify_cellbox(data, reaction, world_x, xx, yy, width, height):
    cellbox_cells = set()   # Originally created an empty list with []
    
    # Convert our cell box into a list of cell IDs
    for ypos in range(yy,yy+height):
        for xpos in range(xx,xx+width):
            cell_id = ypos * world_x + xpos
            cellbox_cells.add(cell_id)   # Originally we used cellbox_cells.append()
    
    # Iterate over our data per update
    # The iloc is giving us every row and all columns 2 and after
    for update_ndx, update_data in data.iterrows():
        update = update_data.loc['update']   #update_data['update'] also works here
        # Iterate over each cell in the world.  Cell values start at column 2.
        for cell_id, cell_value in enumerate(update_data.iloc[2:]):
            if cell_id not in cellbox_cells and cell_value > 0:
                msg = f'At update {update}, cell_id {cell_id} has a value of {cell_value} '
                msg += f'and is outside the box for reaction {reaction}'
                raise Exception(msg)
    print(f'{reaction} is all inside the cellbox.')



class ORectangle( Rectangle ):
    """
    ORectangle is an offset rectangle to deal with the fact that
    imshow-style plots have position centers in the middle of a
    box.  We want to move it so that they are on an edge
    
    It simply creates a new Rectangle with an offset.  (Rectangle is the parent class.)
    """
    #            ---positional--  ----keyword------
    # ORectangle( (x,y), 10, 20 , ec='cyan', fc='none')
    def __init__(self, xy, *args, **kw):  
        '''
        We're going to change some of our arguments before we pass them
        to to the real Rectangle.
        
        :param xy: tuple of x and y coordinates
        :param *args: any remaining positional arguments passed (stored as a list)
        :param **kw: any named arguments passed (stored as a dictionary)
        '''
        xx, yy = xy    # Unpack the tuple we're given for coordinates
        xx = xx - 0.5  # And offset them
        yy = yy - 0.5
        Rectangle.__init__( self, (xx,yy), *args, **kw )  # Initialize our parent Rectangle class


Test 0
======

Reactions:

    NOT cellbox=10,10,20,20
    NAND cellbox=30,30,20,20

World:

    X=60
    Y=60

In [ ]:
# The data for our experiment is in path
path = 'test0/data/curr-reaction-count.dat'
data = load_reaction_grid(path)

cellboxes = {
    'NOT': {
        'xx':10,
        'yy':10,
        'width':20,
        'height':20
    },
    'NAND': {
        'xx':30,
        'yy':30,
        'width':20,
        'height':20
    }
}

#For each reaction, check to see whether any reactions
#are performed outside of the cell box.  We're passing
#the cellbox arguments as a keyword dictionary so we can
#type them in one place and use a loop to iterate through them
#If there is an error, an exception will be thrown
for reaction in np.unique(data['reaction']):
    reaction_data = data[data['reaction']==reaction]  # Grab just the data for a single reaction
    verify_cellbox(reaction_data, reaction, world_x=60, **cellboxes[reaction])

plt.subplot(1,2,1)  #Subplot with one row, two columns, and we're going to write to the first

not_data = data.iloc[-2,2:]  #(I)ndex locating, second from last row, columns 2 to end
not_data = not_data.values.astype('float').reshape(60,60)  # Reshape to fit our world dimensions
plt.imshow(not_data, interpolation='nearest')

# Rectangle at (10,10) with width 20 and height 20
not_box = ORectangle( (10,10), 20, 20, edgecolor='cyan', facecolor='none')  
plt.gca().add_artist(not_box)  # For the current axes (gca), add the rectangle
plt.title('NOT')


plt.subplot(1,2,2)  # We're writing to the second axes
nand_data = data.iloc[-1,2:]  #(I)ndex locating, last row, columns 2 to end
nand_data = nand_data.values.astype('float').reshape(60,60)  
plt.imshow(nand_data, interpolation='nearest')
nand_box = ORectangle( (30,30), 20, 20, ec='cyan', fc='none' )  #ec = 'edgecolor' and fc = 'facecolor'
plt.gca().add_artist(nand_box)
plt.title('NAND')

Test1
====

Assymetrical testing.  Here we have boxes that aren't the same dimensions in x and y.  The world isn't either.

Reactions:

    NOT cellbox=10,5,30,20
    NAND cellbox=30,20,10,30

World:

    X=50
    Y=70

In [ ]:
# The data for our experiment is in path
path = 'test1/data/curr-reaction-count.dat'
data = load_reaction_grid(path)
world_x = 50
world_y = 70
grid_shape = (world_y,world_x)

cellboxes = {
    'NOT': {
        'xx':10,
        'yy':5,
        'width':30,
        'height':20
    },
    'NAND': {
        'xx':30,
        'yy':20,
        'width':10,
        'height':30
    }
}

reactions = np.unique(data['reaction'])
num_reactions = len(reactions)

fig = plt.figure()
for index, reaction_name in enumerate(reactions):
    RBOX = cellboxes[reaction_name]
    reaction_data = data[data['reaction']==reaction_name]  # Grab just the data for a single reaction
    verify_cellbox(reaction_data, reaction_name, world_x, **RBOX)
    
    plt.subplot(1,num_reactions,index+1)  #Subplot with one row, two columns, and we're going to write to the first
    r_data = reaction_data.iloc[-1,2:]  #(I)ndex locating, last row, columns 2 to end
    r_data = r_data.values.astype('float').reshape(grid_shape)  # Reshape to fit our world dimensions
    plt.imshow(r_data, interpolation='nearest')
    
    r_box = ORectangle( (RBOX['xx'],RBOX['yy']), RBOX['width'], RBOX['height'], edgecolor='cyan', facecolor='none')  
    plt.gca().add_artist(r_box)  # For the current axes (gca), add the rectangle
    plt.title(reaction_name)
    
